# Aircraft maintainance problem - EA demo

In [ ]:
import pandas as pd
import numpy as np
import datetime
import random
import sys
import os

## PlaneModel defines a set of useful classes

In [ ]:
from  PlaneModel import job,aircraft,staff,problem

## Load Problem instance from Excel

Check that cells in the spreadsheet are a 'general' format. Numbers formatted as strings can cause issues

In [ ]:
file = './Test-Data/Example1.xlsx'
planes = pd.read_excel(file,  sheet_name = 0)
work_packages = pd.read_excel(file, sheet_name = 1, index_col = 0)
people = pd.read_excel(file,  sheet_name = 2, index_col = 0)

In [ ]:
planes

## Populate Model

The data from Pandas (itself from Excel) is turned into a set of objects (as defined in the classes above)
The variable instance is an instance of Problem and contains collections of Staff, Aircraft and Jobs


In [ ]:
instance = problem(people,planes,work_packages)

## Generate random genome

In [ ]:

def randSol():
    instance.reset()
    availStaff = instance.getListStaff()
    
    allJobs = instance.getListJobs()
    sol = []
    for j in allJobs:
        theJob = instance.jobs[j]
        c=1
        for c in range(len(theJob.notAllocated)):
            
            rStaff = random.choice(availStaff)
            while not theJob.check(instance.staff[rStaff]):
                rStaff = random.choice(availStaff)
                
            code= j+":"+str(c)
            t = [rStaff,code]
            sol.append(t)
            c=c+1
    random.shuffle(sol)
    return sol

## Evaluate Genome

In [ ]:
def evaluate(sol):
    instance.reset()
    for g in sol:
        j=g[1].split(":")[0]
        instance.allocate(instance.staff[g[0]],instance.jobs[j])

    instance.validate()
    unalloc =instance.getUnallocated()
    over = 0
    countOver =0
    for ac in instance.aircraft:
        airc = instance.aircraft[ac]
        o = airc.over()
        over = over +o
        if o >0 :
            countOver = countOver + 1
        
    return (unalloc + over),unalloc,countOver,over


## Mutation function

In [ ]:
def mutate(genome):
    
    ch = random.randint(0,3)
    if ch ==1:
        n = random.randint(0,len(genome)-1)
        availStaff = instance.getListStaff()
        
        j =  genome[n][1].split(":")[0]
        theJob = instance.jobs[j]
        rStaff = random.choice(availStaff)
        while not theJob.check(instance.staff[rStaff]):
                rStaff = random.choice(availStaff)
        genome[n][0]= rStaff
        

    if ch==2:
        x = random.randint(0,len(genome)-1)
        y = random.randint(0,len(genome)-1)
        t = genome[x]
        genome.pop(x)
        genome.insert(y,t)
    if ch == 3:
        genome = timeMutate(genome)
    return genome


def timeMutate(genome):
    tl = datetime.timedelta(minutes=0)
    gene = None
    
    for c  in range(10):        
        x = random.randint(0,len(genome)-1)        
        g = genome[x]
        jb = g[1].split(':')[0]
        if instance.jobs[jb].duration > tl:
            tl = instance.jobs[jb].duration
            gene = x
    
    t = genome[gene]
    genome.pop(gene)
    genome.insert(0,t)
    return genome



## Copy genome

This is a deep copy

In [ ]:
def copyG(genome):
    n = []
    for g in genome:
        n.append(g.copy())
    
    return n

# Simple EA

## Crossover (XO) function

In [ ]:
def contains(genome, jCode):
    for j in genome:
        if j[1] == jCode:
            return True
    return False


def xo(pA,pB):
    if len(pA) != len(pB):
        print("Parent len mismatch")
  
    child = []
    for c in  range(len(pA)):
        if not contains(child,pA[c][1]):
                child.append(pA[c].copy())
        if not contains(child,pB[c][1]):
                child.append(pB[c].copy())
    
    return child


# EA

In [ ]:
def tour(pop):
    p1 = random.choice(pop)
    p2 = random.choice(pop)
    
    if p1[0] < p2[0]:
        return p1
    else:
        return p2
        
def rip(pop):
    p1 = random.choice(pop)
    p2 = random.choice(pop)
    
    if p1[0] >  p2[0]:
        return p1
    else:
        return p2
          
    
pop_size = 1500 
budget = 100000

best = None

pop = []
for c in range(0,pop_size):

    i = randSol()
    f= evaluate(i)[0]
    p= (f,i)
    pop.append(p)
    if len(pop)==1:
        best=p
    
    
    if f < best[0]:
        best=p
        
print("Init")
print(best[0])
evals =0
while (evals < budget):
    evals = evals +1
    if random.choice([True,False]):
        parent = tour(pop)
        ng = copyG(parent[1])
    else:
        ng = xo(tour(pop)[1],tour(pop)[1])
        
    mutate(ng)
    nf = evaluate(ng)[0]
    child = (nf,ng)

    toGo = rip(pop)
    if toGo[0] > child[0]:
        pop.remove(toGo)
        pop.append(child)
        if child[0] < best[0]:
            best = (child[0],copyG(child[1])) 
            instance.reset()
            r = evaluate(best[1])
            print("Evals: " + str(evals))
            print("Improved (" +str(r[0])+") Missing staff = "+ str(r[1]) + " Late = "+ str(r[2]) + " Late mins = " + str(r[3]))
              
print("Done :" +str(best[0]))


In [ ]:
print("Final Result:")
instance.reset()
r = evaluate(best[1])
print("Fitness = " +str(r[0])+" Missing staff = "+ str(r[1]) + " Late Departures= "+ str(r[2]) + " Late mins = " + str(r[3]))
print("\nPlan:\n\n"+str(instance))

              

### Fine-tuning the EA outputs

In [ ]:
for run in range(10):
    pop_size = 1500 
    budget = 100000

    best = None

    pop = []
    for c in range(pop_size):
        i = randSol()
        f = evaluate(i)[0]
        p = (f, i)
        pop.append(p)
        if len(pop) == 1:
            best = p
        if f < best[0]:
            best = p

    print(f"Run {run + 1} - Init")
    print(f"Initial best: {best[0]}")

    evals = 0
    fitness_All = []
    while evals < budget:
        evals += 1
        if random.choice([True, False]):
            parent = tour(pop)
            ng = copyG(parent[1])
        else:
            ng = xo(tour(pop)[1], tour(pop)[1])
        
        mutate(ng)
        nf = evaluate(ng)[0]
        child = (nf, ng)

        toGo = rip(pop)
        if toGo[0] > child[0]:
            pop.remove(toGo)
            pop.append(child)
            if child[0] < best[0]:
                best = (child[0], copyG(child[1]))
                instance.reset()
                r = evaluate(best[1])
                print(f"Evals: {evals}")
                print(f"Improved ({r[0]}) Missing staff = {r[1]} Late = {r[2]} Late mins = {r[3]}")

    fitness_All.append(best[0])
    print(f"Run {run + 1} - Done: {best[0]}\n")
    
    # compute the average of fitness values in the population for 10 iterations
    avg_fitness = np.mean(fitness_All)
    print(f"Average fitness over 10 runs: {avg_fitness}")